In [24]:
!pip install -r requirements.txt

In [25]:
import os

os.environ["OPENAI_API_KEY"] = 'Enter your api key here.'

# Math chain

We can use Math Chain to do complex word math problems.

In [26]:
from langchain import OpenAI, LLMMathChain

llm = OpenAI(temperature=0)
llm_math = LLMMathChain.from_llm(llm, verbose=True)

llm_math.run("What is 13 raised to the .3432 power?")



> Entering new  chain...
What is 13 raised to the .3432 power?
```text
13**.3432
```
...numexpr.evaluate("13**.3432")...

Answer: 2.4116004626599237
> Finished chain.


'Answer: 2.4116004626599237'

# Bash chain

we can use LLMs and a bash process to perform simple filesystem commands.

In [27]:
from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

text = "Please write a bash script that prints 'Hello World' to the console."

bash_chain = LLMBashChain.from_llm(llm, verbose=True)

bash_chain.run(text)



> Entering new  chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
echo "Hello World"
```
Code: ['echo "Hello World"']
Answer: "Hello World"

> Finished chain.


'"Hello World"\r\n'

# Customize Prompt

You can also customize the prompt that is used. Here is an example prompting to avoid using the 'echo' utility

In [28]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm_bash.prompt import BashOutputParser

_PROMPT_TEMPLATE = """If someone asks you to perform a task, your job is to come up with a series of bash commands that will perform the task. There is no need to put "#!/bin/bash" in your answer. Make sure to reason step by step, using this format:
Question: "copy the files in the directory named 'target' into a new directory at the same level as target called 'myNewDirectory'"
I need to take the following actions:
- List all files in the directory
- Create a new directory
- Copy the files from the first directory into the second directory
```bash
ls
mkdir myNewDirectory
cp -r target/* myNewDirectory
Do not use 'echo' when writing the script.

That is the format. Begin! Question: {question}"""

In [29]:
PROMPT = PromptTemplate( input_variables=["question"], template=_PROMPT_TEMPLATE, output_parser=BashOutputParser(), )

In [30]:
bash_chain = LLMBashChain.from_llm(llm, prompt=PROMPT, verbose=True)

# text = "Please write a bash script that prints 'Hello World' to the console."
text = "How to create a text file in the directory 'myfolder' with bash."

bash_chain.run(text)



> Entering new  chain...
How to create a text file in the directory 'myfolder' with bash.

mkdir myfolder
touch myfolder/myfile.txt

OutputParserException: Failed to parse bash output. Got: mkdir myfolder
touch myfolder/myfile.txt

# Persistent Terminal

By default, the chain will run in a separate subprocess each time it is called. This behavior can be changed by instantiating with a persistent bash process.

In [ ]:
from langchain.utilities.bash import BashProcess


persistent_process = BashProcess(persistent=True)
bash_chain = LLMBashChain.from_llm(llm, bash_process=persistent_process, verbose=True)

text = "List the current directory then move up a level."

bash_chain.run(text)

In [ ]:
# Run the same command again and see that the state is maintained between calls
bash_chain.run(text)